## Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import matplotlib.pyplot as plt

## Data Query

In [2]:
url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json"  # Replace with the actual API endpoint
params = {
    'format': 'json',  # Output response format
    'api_key': '***',  # Your developer API key
    'status': 'all',  # Return stations that match the given status
    # 'access': 'public',  # Return stations with the given access type
    'fuel_type': 'ELEC', # Return stations that supply any of the given fuel types
    'state': 'CA',
    'country': 'US',
    
}

response = requests.get(url, params=params)
if response.status_code == 200:

    data = response.json()

In [3]:
dmv_data = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2024.csv")

In [4]:
sdge_zips_data = pd.read_excel("data/SDGE Service Territory Zip Code List Q2-2021.xlsx")

## Data Pre-Processing

In [5]:
sdge_zips = list(sdge_zips_data['ZIP_CODE'])
print(len(sdge_zips))

In [6]:
# CS - Charging Stations
cs = pd.DataFrame(data['fuel_stations'])
cs = cs[cs['zip'] != 'CA']
cs['zip'] = cs['zip'].astype(int)
cs = cs[cs['zip'].isin(sdge_zips)]
print(cs.shape)
cs.head()

In [7]:
dmv = dmv_data.copy()
dmv = dmv[dmv['ZIP Code']!="OOS"]
dmv['ZIP Code'] = dmv['ZIP Code'].astype(int)
dmv = dmv[dmv['ZIP Code'].isin(sdge_zips)]
print(dmv.shape)
dmv.head()

In [8]:
dmv.columns

## Pre-EDA

In [9]:
# Calculate EV for each zip code (in SDGE territory)
# And merge with dmv dataset for a new table
zip_totals = dmv.groupby('ZIP Code')['Vehicles'].sum().reset_index(name='Lambda')
ev_totals = dmv[dmv['Fuel']=="Battery Electric"].groupby('ZIP Code')['Vehicles'].sum().reset_index(name='EV Lambda')
df = dmv.merge(zip_totals, on='ZIP Code', how='left')
df = df.merge(ev_totals, on='ZIP Code', how='left')

# Count the number of charging stations per ZIP code
cs_counts = cs.groupby('zip').size().reset_index(name='ChargingStations')
df = df.merge(cs_counts, left_on='ZIP Code', right_on='zip', how='left')
df.drop(columns=['zip'], inplace=True)
df['ChargingStations'] = df['ChargingStations'].fillna(0).astype(int)

df['CSperEV'] = df['ChargingStations']/df['EV Lambda']

df.head()

## EDA

In [10]:
from sklearn.preprocessing import StandardScaler

zips = df.groupby('ZIP Code')[['EV Lambda', 'ChargingStations', 'CSperEV']].mean()#.sort_values(by='CSperEV', ascending=True)

scaler = StandardScaler()
zips['CSperEVscaled'] = scaler.fit_transform(zips[['CSperEV']])

# Calculate the mean and standard deviation
mean = zips['CSperEVscaled'].mean()
std = zips['CSperEVscaled'].std()

# Cap the values at 3 standard deviations
cap_upper = mean + 3 * std
cap_lower = mean - 3 * std

# Apply the capping
zips['CSperEVscaled'] = zips['CSperEVscaled'].clip(lower=cap_lower, upper=cap_upper)
# zips['CSperEV_log'] = np.log1p(zips['CSperEV'])

zips.head()

### Map

In [11]:
import geopandas as gpd
import folium

# Load the ZCTA shapefile
zcta = gpd.read_file("data/California_Census_ZIP_Code_Tabulation_Areas_-3654693237815920706.zip")

# Filter for California ZIP codes (ZCTAs starting with '92' cover Southern CA)
zcta = zcta[zcta['ZCTA5'].str.startswith('92')]

# Ensure ZIP codes are integers for merging
zcta['ZCTA5'] = zcta['ZCTA5'].astype(int)

# zcta.head()

In [12]:
# Merge ZCTA shapefile with zips
zcta = zcta.rename(columns={'ZCTA5': 'ZIP Code'})
merged = zcta.merge(zips, on='ZIP Code', how='inner')

# Cap ratio to limit outliers
merged.loc[merged['CSperEV'] > 0.5, 'CSperEV'] = 0.5

# merged.head(_

In [13]:
from folium import Choropleth

# Create a base map centered on San Diego
m = folium.Map(location=[32.7157, -117.1611], zoom_start=10)

# Custom binning with heavier weights towards 0 and capping the upper limit at 0.5
bins = [0, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5]  # Custom bins
colors = ['#f7f7f7', '#fee5d9', '#fcae91', '#fb6a4a', '#e31a1c', '#bd0026', '#800026', '#660000']  # Color map for bins

# Add choropleth map with custom bins
folium.Choropleth(
    geo_data=merged,  # GeoJSON data
    data=merged,  # Dataframe containing the 'CSperEV' column
    columns=['ZIP Code', 'CSperEV'],  # Specify the column names
    key_on="feature.properties.ZIP Code",  # Specify the key in GeoJSON
    fill_color='YlOrRd',  # You can customize the color scheme here
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Charging Stations per EV",
    bins=bins,  # Custom bins for scaling
    reset=True,
    highlight=True
).add_to(m)

print("SDGE Zip Codes by ratio of EV Chargers to registered EV Vehicles")

# Save the map
m.save("charging_stations_by_zip_weighted.html")
m